In [1]:
import polars as pl
import numpy as np
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv("Kay_Dataset.csv", encoding='latin1')
df = df.drop(df.columns[0], axis=1) #Enelever code community_area pour juste laisser le string
df = df.drop(df.columns[1], axis=1) #Enlever la description car ca devient le seul critere de l'arbre
x_features = df.columns.delete(1)
df.head()

,primary_type,Domestic,location_description,Date,Comm_Area,Hardship_Score,Crowded_Housing_Per,Below_Poverty_Household_Income,Unemployment_Rate,No_High_School_Diploma,Under_18_over_64,Per_Capita_Income
0,OFFENSE AND KIDNAPPING,True,APARTMENT,Night,Gage Park,65.3,12.1,18.5,13.7,22.4,39.0,13605
1,OFFENSE AND KIDNAPPING,True,APARTMENT,Night,Gage Park,65.3,12.1,18.5,13.7,22.4,39.0,13605
2,OFFENSE AND KIDNAPPING,True,RESIDENCE,Night,South Lawndale,70.6,13.6,30.3,8.7,28.8,35.3,11345
3,PUBLIC ORDER CRIMES,False,APARTMENT,Night,South Chicago,54.9,3.4,28.9,20.3,11.6,41.1,16996
4,THEFT,False,RESIDENCE,Night,Irving Park,32.3,3.5,10.3,5.8,10.3,30.2,31043


In [3]:
X = df.drop('primary_type', axis=1)
y = df['primary_type']
y

0          OFFENSE AND KIDNAPPING
1          OFFENSE AND KIDNAPPING
2          OFFENSE AND KIDNAPPING
3             PUBLIC ORDER CRIMES
4                           THEFT
                    ...          
1045246       PUBLIC ORDER CRIMES
1045247       PUBLIC ORDER CRIMES
1045248       PUBLIC ORDER CRIMES
1045249                     THEFT
1045250       PUBLIC ORDER CRIMES
Name: primary_type, Length: 1045251, dtype: object

In [4]:
primary = LabelEncoder()
y_encoded = primary.fit_transform(y)

y_encoded

array([2, 2, 2, ..., 3, 5, 3])

In [5]:
onehot_encoder = OneHotEncoder()
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', onehot_encoder, ['location_description', 'Date', 'Comm_Area'])
    ],
    remainder='passthrough' 
)
X_encoded = column_transformer.fit_transform(X)
all_encoded_feature_names = column_transformer.get_feature_names_out()
encoded_feature_names = column_transformer.named_transformers_['cat'].get_feature_names_out(['location_description', 'Date', 'Comm_Area'])
X_encoded_df = pd.DataFrame(X_encoded.toarray(), columns=all_encoded_feature_names)
X_encoded_df.to_csv('X_OneHotEncoder.csv', index=False)
X_encoded_df


,cat__location_description_ABANDONED BUILDING,cat__location_description_AIRCRAFT,cat__location_description_AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA,cat__location_description_AIRPORT BUILDING NON-TERMINAL - SECURE AREA,cat__location_description_AIRPORT EXTERIOR - NON-SECURE AREA,cat__location_description_AIRPORT EXTERIOR - SECURE AREA,cat__location_description_AIRPORT PARKING LOT,cat__location_description_AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA,cat__location_description_AIRPORT TERMINAL LOWER LEVEL - SECURE AREA,cat__location_description_AIRPORT TERMINAL MEZZANINE - NON-SECURE AREA,...,cat__Comm_Area_West Town,cat__Comm_Area_Woodlawn,remainder__Domestic,remainder__Hardship_Score,remainder__Crowded_Housing_Per,remainder__Below_Poverty_Household_Income,remainder__Unemployment_Rate,remainder__No_High_School_Diploma,remainder__Under_18_over_64,remainder__Per_Capita_Income
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,65.3,12.1,18.5,13.7,22.4,39.0,13605.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,65.3,12.1,18.5,13.7,22.4,39.0,13605.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,70.6,13.6,30.3,8.7,28.8,35.3,11345.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,54.9,3.4,28.9,20.3,11.6,41.1,16996.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,32.3,3.5,10.3,5.8,10.3,30.2,31043.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,70.5,4.0,42.4,34.1,16.1,41.5,12777.0
1045247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,52.6,2.2,24.0,24.3,9.6,41.8,19755.0
1045248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,38.4,2.0,13.7,12.3,8.1,37.8,27488.0
1045249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,47.9,2.9,26.1,19.0,8.0,37.9,21416.0


In [6]:
x_array = X_encoded_df.to_numpy()
x_array

array([[0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 2.2400e+01, 3.9000e+01,
        1.3605e+04],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 2.2400e+01, 3.9000e+01,
        1.3605e+04],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 2.8800e+01, 3.5300e+01,
        1.1345e+04],
       ...,
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 8.1000e+00, 3.7800e+01,
        2.7488e+04],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 8.0000e+00, 3.7900e+01,
        2.1416e+04],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 8.2000e+00, 3.6700e+01,
        1.9117e+04]])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(x_array, y_encoded, test_size = 0.3) 
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=1).fit(X_train, y_train)

crime_names = list(primary.classes_)
x_features = X_encoded_df.columns
features = list(x_features)


plt.figure(figsize=(60, 50)) 
tree.plot_tree(clf, class_names=crime_names, feature_names=features, filled=True)
plt.savefig('decision_tree_high_res.png', dpi=300, bbox_inches='tight') 
# plt.show()
plt.close()



print("\n***** Learning performances: *****") 
# make predictions
y_valid = clf.predict(X_train)
#evaluate results
print("Confusion Matrix: ") 
print(confusion_matrix(y_train, y_valid))
print ("Accuracy : ", accuracy_score(y_train, y_valid)*100) 

print("\n***** Prediction performances: *****") 
# make predictions
y_pred = clf.predict(X_test)
#evaluate results
print("Confusion Matrix: ") 
print(confusion_matrix(y_test, y_pred))
print ("Accuracy : ", accuracy_score(y_test,y_pred)*100) 


***** Learning performances: *****
Confusion Matrix: 
[[127962      0    960      0      0 156694]
 [    76      0      0      0      0    320]
 [ 27199      0   1619      0      0  26748]
 [  5799      0     64      0      0  61222]
 [  2413      0     32      0      0   4987]
 [ 35493      0    109      0      0 279978]]
Accuracy :  55.975535586155054

***** Prediction performances: *****
Confusion Matrix: 
[[ 54711      0    459      0      0  67626]
 [    28      0      1      0      0    177]
 [ 11167      0    685      0      0  11463]
 [  2438      0     36      0      0  26282]
 [  1069      0      9      0      0   2127]
 [ 14982      0     42      0      0 120274]]
Accuracy :  56.02150674796541


In [8]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_array, y_encoded, test_size = 0.3) 
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)

# Make predictions on the training set
y_train_pred = dummy_clf.predict(X_train)
# Make predictions on the test set
y_test_pred = dummy_clf.predict(X_test)

# Evaluate the accuracy of the dummy classifier on training and test sets
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Baseline Training Accuracy: {train_accuracy:.2f}")
print(f"Baseline Test Accuracy: {test_accuracy:.2f}")

# For a more comprehensive report, you can also look at precision, recall, and F1-score
print("\nClassification Report for the Baseline Model (Training):")
print(classification_report(y_train, y_train_pred))

print("\nClassification Report for the Baseline Model (Test):")
print(classification_report(y_test, y_test_pred))



Baseline Training Accuracy: 0.43
Baseline Test Accuracy: 0.43

Classification Report for the Baseline Model (Training):


C:\Users\K-1\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\K-1\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\K-1\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00    285747
           1       0.00      0.00      0.00       431
           2       0.00      0.00      0.00     55338
           3       0.00      0.00      0.00     66902
           4       0.00      0.00      0.00      7401
           5       0.43      1.00      0.60    315856

    accuracy                           0.43    731675
   macro avg       0.07      0.17      0.10    731675
weighted avg       0.19      0.43      0.26    731675


Classification Report for the Baseline Model (Test):
              precision    recall  f1-score   support

           0       0.00      0.00      0.00    122665
           1       0.00      0.00      0.00       171
           2       0.00      0.00      0.00     23543
           3       0.00      0.00      0.00     28939
           4       0.00      0.00      0.00      3236
           5       0.43      1.00      0.60    135022

    accuracy           

C:\Users\K-1\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\K-1\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\K-1\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
#X_train, X_test, y_train, y_test = train_test_split(x_array, y_encoded, test_size = 0.3) 
#RdmF = RandomForestClassifier(n_estimators=100, random_state=0).fit(X_train, y_train);

#print("\n***** Learning performances: *****") 
## make predictions
#y_valid = RdmF.predict(X_train)
##evaluate results
#print("Confusion Matrix: ") 
#print(confusion_matrix(y_train, y_valid))
#print ("Accuracy : ", accuracy_score(y_train, y_valid)*100) 

#print("\n***** Prediction performances: *****") 
## make predictions
#y_pred = RdmF.predict(X_test)
##evaluate results
#print("Confusion Matrix: ") 
#print(confusion_matrix(y_test, y_pred))
#print ("Accuracy : ", accuracy_score(y_test,y_pred)*100) 